In [ ]:
import os
import googlemaps
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv

In [ ]:
load_dotenv()
API_KEY = os.getenv('GOOGLE_MAPS_API_KEY')

# Initialize the Google Maps client
gmaps = googlemaps.Client(key=API_KEY)

# Function to get popular places in Riyadh
def get_popular_places(location='Riyadh', radius=50000, place_type='tourist_attraction'):
    places_result = gmaps.places_nearby(location=location, radius=radius, type=place_type)
    places = []
    while places_result:
        places.extend(places_result['results'])
        if 'next_page_token' not in places_result:
            break
        else:
            next_page_token = places_result['next_page_token']
            places_result = gmaps.places_nearby(location=location, radius=radius, type=place_type, page_token=next_page_token)
    return places

# Function to get traffic data
def get_traffic_data(origin, destination):
    try:
        directions_result = gmaps.directions(origin, destination, departure_time='now')
        route = directions_result[0]['legs'][0]
        traffic_data = {
            'route_id': f"{origin}_{destination}_{datetime.now().strftime('%Y%m%d%H%M%S')}",
            'origin': origin,
            'destination': destination,
            'distance_km': route['distance']['value'] / 1000,
            'duration_min': route['duration']['value'] / 60,
            'duration_in_traffic_min': route['duration_in_traffic']['value'] / 60,
            'departure_time': datetime.now().isoformat(),
            'traffic_condition': 'N/A',  # This needs to be determined based on speed and delay
            'speed_kmh': (route['distance']['value'] / 1000) / (route['duration_in_traffic']['value'] / 3600),
            'delay_min': (route['duration_in_traffic']['value'] - route['duration']['value']) / 60,
            'timestamp': datetime.now().isoformat(),
            'day_of_week': datetime.now().strftime('%A'),
            'time_of_day': datetime.now().strftime('%p')  # AM/PM format for time of day
        }
        return traffic_data
    except Exception as e:
        print(f"Error fetching data for {origin} to {destination}: {e}")
        return None

# Get popular places in Riyadh
location = {'lat': 24.7136, 'lng': 46.6753}  # Coordinates for Riyadh
popular_places = get_popular_places(location=location)
place_names = [place['name'] + ', Riyadh' for place in popular_places]

# Generate pairs of origin and destination
directions = [(origin, destination) for origin in place_names for destination in place_names if origin != destination]

# Initialize data lists
routes_list = []
traffic_conditions_list = []
points_of_interest_list = []
traffic_data_list = []

# Collect data
poi_id_counter = 1
for origin, destination in directions:
    route_data = get_traffic_data(origin, destination)
    if route_data:
        # Add route to routes_list
        route_id = route_data['route_id']
        if route_id not in [route['route_id'] for route in routes_list]:
            routes_list.append({
                'route_id': route_id,
                'origin': origin,
                'destination': destination,
                'distance_km': route_data['distance_km']
            })
        
        # Add traffic condition to traffic_conditions_list
        traffic_condition_id = len(traffic_conditions_list) + 1
        traffic_conditions_list.append({
            'condition_id': traffic_condition_id,
            'route_id': route_id,
            'duration_min': route_data['duration_min'],
            'duration_in_traffic_min': route_data['duration_in_traffic_min'],
            'departure_time': route_data['departure_time'],
            'speed_kmh': route_data['speed_kmh'],
            'delay_min': route_data['delay_min'],
            'timestamp': route_data['timestamp'],
            'day_of_week': route_data['day_of_week'],
            'time_of_day': route_data['time_of_day']
        })
        
        # Add points of interest to points_of_interest_list
        for place in popular_places:
            poi_name = place['name']
            if poi_name not in [poi['poi_name'] for poi in points_of_interest_list]:
                points_of_interest_list.append({
                    'poi_id': poi_id_counter,
                    'route_id': route_id,
                    'poi_name': poi_name,
                    'poi_type': place.get('types', ['unknown'])[0],
                    'poi_distance_m': place.get('distance', 'N/A'),  # Placeholder for distance
                    'poi_impact': 'N/A'  # Placeholder for impact
                })
                poi_id_counter += 1
        
        # Add traffic data to traffic_data_list
        traffic_data_list.append({
            'data_id': len(traffic_data_list) + 1,
            'route_id': route_id,
            'condition_id': traffic_condition_id,
            'poi_id': poi_id_counter - 1  # Assuming the last added POI is relevant
        })
        
        if len(traffic_data_list) >= 20000:
            break

# Convert to DataFrame
routes_df = pd.DataFrame(routes_list)
traffic_conditions_df = pd.DataFrame(traffic_conditions_list)
points_of_interest_df = pd.DataFrame(points_of_interest_list)
traffic_data_df = pd.DataFrame(traffic_data_list)

# Display DataFrames
print("Routes:")
print(routes_df.head())
print("\nTraffic Conditions:")
print(traffic_conditions_df.head())
print("\nPoints of Interest:")
print(points_of_interest_df.head())
print("\nTraffic Data:")
print(traffic_data_df.head())

# Save data to CSV
routes_df.to_csv('routes.csv', index=False)
traffic_conditions_df.to_csv('traffic_conditions.csv', index=False)
points_of_interest_df.to_csv('points_of_interest.csv', index=False)
traffic_data_df.to_csv('traffic_data.csv', index=False)
